# DS 4300: Book Recommendation Engine
## Sara Adra, Anika Das, Mirah Gordon, Genny Jawor

This notebook is meant to serve as the home of all data loading and cleaning from the raw book, author, and review json files.

The code here includes:
* loading in data to mongo and filtering data based on set parameters (average rating, number of ratings, and number of text reviews) 
    * this was done exclusively in a local connection to mongo
* connecting to the mongo client
* cleaning the book data
* finding the top reviews for each book
* finding the author name for each book
* running sentiment analysis on reviews to find the most common words
* exporting the data needed to create neo4j graph

### Loading Data

In [1]:
# import pymonogo to connect to the database and our collections
import pymongo
from pymongo import MongoClient
import pprint
import operator
# create a mongo client
client = MongoClient()
# connect the client to the local host
client = MongoClient('localhost', 27017)

### List of Mongo Commands Used on Home Computer

These commands were run through the mongodb shell on a local computer instead of through pymongo to increase the efficiency of the command. 

**Loading Data Into Mongo Collections**
* Reviews
    - /Users/mirahgordon/documents/MongoDB/bin/mongodb-tools/bin/mongoimport --db demo --collection reviews --file $HOME/data/reviews/goodreads_reviews.json
    
* Books
    - /Users/mirahgordon/documents/MongoDB/bin/mongodb-tools/bin/mongoimport --db demo --collection books --file $HOME/data/books/goodreads_books.json
    
* Authors 
    - /Users/mirahgordon/documents/MongoDB/bin/mongodb-tools/bin/mongoimport --db demo --collection authors --file $HOME/data/authors/goodreads_book_authors.json

**Deleting Unnecessary Fields From Collections**
* Books
    - Fields to delete: 
        - asin, country_code, edition_information, format, image_url, is_ebook, isbn, isbn13, kindle_asin, language_code, link, publication_day, publication_month, series, similar_books, url, work_id
    
    - Command:
        - db.books.updateMany( {}, { $unset: {asin: "", country_code: "", edition_information: "", format: "", image_url: "", is_ebook: "", isbn: "", isbn13: "", kindle_asin: "", link: "", publication_day: "", publication_month: "", series: "", similar_books: "", url: "", work_id: "" }} )
    
* Reviews
    - Fields to delete:
        - date_added, date_updated, read_at, started_at, user_id
    - Command:
        - db.reviews.updateMany( {}, { $unset: {date_added: "", date_updated: "", read_at: "", started_at: "", user_id: "" }} )

**Filtering Books by Average Rating**
* db.books.remove( { average_rating: { $lt: '4.00' } } )

**Filtering Books by Number of Text Reviews**
* db.books.remove( { text_reviews_count: { $lt: '5000.0' } } )

**Filtering Books by Number of Ratings**
* db.books.remove( { ratings_count: { $lt: '5000.0' } } )

**Create New Reviews Index using book_id**
* db.reviews.createIndex( { 'book_id': -1 } )

**Create New Authors Index using author_id**
* db.authors.createIndex( { 'author_id': -1 } )

**Setting up Mongo Client**

In [2]:
# find and use the demo database
db = client.demo
# find and use the books collection
books = db.books
# find and use the authors collection
authors = db.authors
# find and use the reviews collection
reviews = db.reviews

In [3]:
# test to ensure the server is connected and will print out the first document in books
for book in books.find().limit(2):
    pprint.pprint(book)

{'_id': ObjectId('6261cc491f034259ef58d8df'),
 'authors': [{'author_id': '4862', 'role': ''}],
 'average_rating': '4.26',
 'book_id': '89376',
 'description': 'What is Heaven really going to be like? What will we look '
                "like? What will we do? Won't Heaven get boring after a "
                'while?\n'
                'We all have questions about what Heaven will be like, and '
                'after 25 years of extensive research, Dr. Randy Alcorn has '
                'the answers.\n'
                'In the most comprehensive and definitive book on Heaven to '
                'date, Randy invites you to picture Heaven the way Scripture '
                'describes it-- a bright, vibrant, and physical New Earth, '
                'free from sin, suffering, and death, and brimming with '
                "Christ's presence, wondrous natural beauty, and the richness "
                'of human culture as God intended it.\n'
                'God has put eternity in our h

In [4]:
# test to ensure the server is connected and will print out the first document in authors
for author in authors.find().limit(1):
    pprint.pprint(author)

{'_id': ObjectId('6261d1778687e60679f430fb'),
 'author_id': '3041852',
 'average_rating': '3.89',
 'name': 'Alfred J. Church',
 'ratings_count': '947',
 'text_reviews_count': '85'}


In [5]:
# test to ensure the server is connected and will print out the first document in reviews
for review in reviews.find().limit(1):
    pprint.pprint(review)

{'_id': ObjectId('6261ce995a5bd73d16c340b1'),
 'book_id': '1995421',
 'n_comments': 0,
 'n_votes': 4,
 'rating': 0,
 'review_id': '7350a30a2f5c785b190d9ebd1c0b4af9',
 'review_text': 'Kevin highly recommended on instagram'}


### Cleaning Book Data

In [6]:
# imports
import pandas as pd
import numpy as np

In [7]:
# save the current list of book documents from mongo (using find command) to a datafram
book_df = pd.DataFrame(list(books.find()))

In [8]:
# save book data frame to a csv 
book_df.to_csv('books_clean.csv')

# open csv into data frame
book_df = pd.read_csv('books_clean.csv')
# drop the _id and duplicate index columns 
book_df.drop(columns=['Unnamed: 0', '_id'], inplace=True)
book_df.head()

,text_reviews_count,popular_shelves,average_rating,description,authors,publisher,num_pages,publication_year,book_id,ratings_count,title,title_without_series
0,566,"[{'count': '6393', 'name': 'to-read'}, {'count...",4.26,What is Heaven really going to be like? What w...,"[{'author_id': '4862', 'role': ''}]",NaN,533.0,NaN,89376,7345,Heaven,Heaven
1,6,"[{'count': '515', 'name': 'to-read'}, {'count'...",4.22,"To Kara's astonishment, she discovers that a p...","[{'author_id': '19158', 'role': ''}]",Seven Seas,216.0,2009.0,6066812,98,All's Fairy in Love and War (Avalon: Web of Ma...,All's Fairy in Love and War (Avalon: Web of Ma...
2,6,"[{'count': '20', 'name': 'to-read'}, {'count':...",4.06,These are the stories that catapulted Superman...,"[{'author_id': '81563', 'role': ''}, {'author_...",DC Comics,272.0,1997.0,707611,51,"Superman Archives, Vol. 2","Superman Archives, Vol. 2"
3,6,"[{'count': '735', 'name': 'to-read'}, {'count'...",4.02,O Alienista e uma celebre obra literaria do es...,"[{'author_id': '22458', 'role': ''}, {'author_...",nshr lwH fkhr,232.0,2005.0,316994,57,روانکاو و داستان‌های دیگر,روانکاو و داستان‌های دیگر
4,9,"[{'count': '46', 'name': 'to-read'}, {'count':...",4.29,"A killer by day and night, Zaire Pearson never...","[{'author_id': '4973079', 'role': ''}]",Jessica Watkins Presents,129.0,2014.0,23272157,51,Beautee and the Beast,Beautee and the Beast


In [9]:
# drop any rows that don't have a description or a number of pages
book_df['description'] = book_df['description'].replace('', np.nan)
book_df = book_df.dropna(axis=0, subset=['description', 'num_pages'])

In [10]:
# creates a df of 100 random books
books_100 = book_df.sample(n=100)

# creates a df of 10,000 books
books_10000 = book_df.sample(n=10000)

In [11]:
# list of author ids for 100 books -- only taking first author (in cases where there are more than one)
books_100['authors'] = books_100['authors'].apply(lambda x: x.split("'")[3])
authorid_100 = books_100['authors'].tolist()

# list of author ids for 10,000 books -- only taking first author (in cases where there are more than one)
books_10000['authors'] = books_10000['authors'].apply(lambda x: x.split("'")[3])
authorid_10000 = books_10000['authors'].tolist()

# list of book ids for 100 books
bookid_100 = books_100['book_id'].tolist()

# list of book ids for 10,000 books
bookid_10000 = books_10000['book_id'].tolist()

### Retrieving Book Reviews

In [12]:
# create dataframe to hold all the top reviews
top_reviews = pd.DataFrame()

# iterate through 
for book in bookid_100:

    # mongo command to find the reviews with a specific book id
    book_reviews = reviews.find( { 'book_id' : str(book) }, { 'book_id':1, 'review_text':1, 'n_votes':1, '_id':0 } )

    # collect all reviews as a list
    top5 = []

    for r in book_reviews:
        top5.append(r)
    
    # sort reviews by number of votes and keep the top 5
    sorted_by_votes = sorted(top5, key=lambda d: d['n_votes'])
    sorted_by_votes = sorted_by_votes[0:5]
        
    # append the top 5 reviews to the dataframe 
    top_reviews = top_reviews.append(sorted_by_votes, ignore_index=True, sort=False)

In [13]:
# set the index as the book id
top_reviews.set_index('book_id', inplace=True)
# drop any row with an empty review
top_reviews['review_text'] = top_reviews['review_text'].replace('', np.nan)
top_reviews.dropna(axis=0, subset=['review_text'], inplace=True)

In [14]:
top_reviews.head()

,review_text,n_votes
book_id,,
15759838,Life's Rhythms is full of beautiful poems. If ...,1
15759838,Life's Rhythms is a great selection of traditi...,1
18634307,lktb jyd f~ b`D lnqT fym ykhS lt`ml m` l Hyper...,0
18634307,Rollo is a manlet beta that doesn't lift but t...,0
18634307,Interesting book on gender communication and r...,0


In [15]:
# save top 5 reviews for our 100 books to a csv
top_reviews.to_csv('top_reviews.csv')

**Repeat the process for the full set of 10,000 books**

In [16]:
# create dataframe to hold all the top reviews
top_reviews_10000 = pd.DataFrame()

for book in bookid_10000:

    # mongo command to find the reviews with a specific book id
    book_reviews = reviews.find( { 'book_id' : str(book) }, { 'book_id':1, 'review_text':1, 'n_votes':1, '_id':0 } )

    # collect all reviews as a list
    top5 = []

    for r in book_reviews:
        top5.append(r)
    
    # sort reviews by number of votes and keep the top 5
    sorted_by_votes = sorted(top5, key=lambda d: d['n_votes'])
    sorted_by_votes = sorted_by_votes[0:5]
        
    # append the top 5 reviews to the dataframe 
    top_reviews_10000 = top_reviews_10000.append(sorted_by_votes, ignore_index=True, sort=False)

In [17]:
# set the index as the book id
top_reviews_10000.set_index('book_id', inplace=True)
# drop any row with an empty review
top_reviews_10000['review_text'] = top_reviews_10000['review_text'].replace('', np.nan)
top_reviews_10000.dropna(axis=0, subset=['review_text'], inplace=True)

In [18]:
top_reviews_10000.head()

,review_text,n_votes
book_id,,
823653,First book in the Camel Club. Composed of elde...,0
823653,The Camel Club is a group of older men who are...,0
823653,I really liked this book just because there we...,0
823653,It could happen ... conspiracy theorists unite...,0
823653,SM,0


In [19]:
# save top 5 reviews for all 10,000 books to a csv
top_reviews_10000.to_csv('top_reviews_10000.csv')

### Finding Book Author

In [20]:
# empty list to hold all author names
author_names = []

# find the name of the author for each book
for aid in authorid_100:
    # mongo command to find the author with a specific author id 
    author = authors.find( { 'author_id': str(aid) }, { 'name':1, '_id':0 } )
    
    for a in author:
        name = a.get('name')
    
    author_names.append(name)
    
# create new author name column
books_100['author_name'] = author_names

In [21]:
# set the index to be the title of the book
books_100.set_index('title', inplace=True)
# drop unused columns of popular shelves and title without series
books_100.drop(columns=['popular_shelves', 'title_without_series'], inplace=True)

In [22]:
"""
function to define the length of a given book
"""
def book_size(row):
    if row['num_pages'] <= 299:
        return "small"
    elif 300 <= row['num_pages'] <= 599: 
        return "medium"
    elif row['num_pages'] > 600:
        return "large"

In [23]:
# apply book size function to book dataframe
books_100['book_size'] = books_100.apply(book_size, axis=1)

In [24]:
books_100.head()

,text_reviews_count,average_rating,description,authors,publisher,num_pages,publication_year,book_id,ratings_count,author_name,book_size
title,,,,,,,,,,,
Life's Rhythms,6,4.00,Ancient Japanese poetry with a modern twist. O...,4788773,Vickie Johnstone,116.0,2012.0,15759838,8,Vickie Johnstone,small
The Rational Male,75,4.32,The Rational Maleis a rational and pragmatic a...,7328259,CreateSpace,300.0,2013.0,18634307,978,Rollo Tomassi,medium
The Tao Te Ching: 81 Verses by Lao Tzu with Introduction and Commentary,7,4.32,"Tao Te Ching translates very roughly as ""the w...",2622245,Watkins Publishing,224.0,2017.0,31921810,7,Lao Tzu,small
Ex-Treme Measures,8,4.33,"Men: You can't live with them, you can't kill ...",5828647,The Wild Rose Press,228.0,2015.0,26345942,8,Mickey J. Corrigan,small
Their Souls Met in Wishton,7,4.64,When the universe sends Liam Kincaid back into...,15386334,Solstice Publishing,202.0,2016.0,30694216,9,Wanda Penalver Bevan,small


In [25]:
# save the most cleaned dataframe to a csv to use with neo4j
books_100.to_csv('books_100.csv')

**Repeat the process for the full set of 10,000 books**

In [26]:
# empty list to hold all author names
author_names = []

# find the name of the author for each book
for aid in authorid_10000:
    # mongo command to find the author with a specific author id 
    author = authors.find( { 'author_id': str(aid) }, { 'name':1, '_id':0 } )
    
    for a in author:
        name = a.get('name')
    
    author_names.append(name)
    
# create new author name column
books_10000['author_name'] = author_names

In [27]:
# set the index to be the title of the book
books_10000.set_index('title', inplace=True)
# drop unused columns of popular shelves and title without series
books_10000.drop(columns=['popular_shelves', 'title_without_series'], inplace=True)

In [28]:
# apply book size function to book dataframe
books_10000['book_size'] = books_10000.apply(book_size, axis=1)

In [29]:
books_10000.head()

,text_reviews_count,average_rating,description,authors,publisher,num_pages,publication_year,book_id,ratings_count,author_name,book_size
title,,,,,,,,,,,
"The Camel Club (Camel Club, #1)",66,4.02,"It exists at the fringes of Washington, D.C., ...",9291,NaN,438.0,2005.0,823653,589,David Baldacci,medium
"Silver Dawn (Wishes, #4.5)",52,4.53,*Silver Dawn - book 4.5 of the Wishes Series*\...,6935697,G.J. Walker-Smith,254.0,2014.0,22046993,734,G.J. Walker-Smith,small
Resurrections (Thin Ice #4),8,4.69,"Life sometimes takes strange twists, for seemi...",3297163,Kirabaco Publishing,245.0,2011.0,13006730,55,K.R. Bankston,small
La Rosa e il Deserto,6,4.11,"Marylya, bella e vivace principessa del Regno ...",14370629,CreateSpace,434.0,2015.0,26637061,9,Lorenza Bartolini,medium
"Il principe (Shadowhunters - Le origini, #2)",63,4.46,"In una Londra vittoriana fosca e inquietante, ...",150038,Mondadori,501.0,2013.0,17315391,749,Cassandra Clare,medium


In [30]:
# save the most cleaned dataframe to a csv to use with neo4j
books_10000.to_csv('books_10000.csv')

### Finding Common Words

In [31]:
# imports for stop words and counter
from collections import Counter
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
set_stopwords = set(stopwords.words('english'))
set_stopwords.update(['book', 'books', 'author', 'story', 'read', "i've"])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mirahgordon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
# compile all reviews for 100 books 
reviews_list = top_reviews['review_text'].tolist()

In [33]:
"""
function to clean the given word (by making it all lower case
letters + removing any trailing punctuation if present)
"""
def clean_word(word):
    # make the word all lowercase letters
    cleaned_word = word.lower()
    
    # check if the last character in the word is a letter or not
    # remove last character if not a letter
    # (ex. ',' '.' '-' ' ')
    if (cleaned_word[-1].isalpha() == False):
         cleaned_word = cleaned_word[:-1]
    
    return cleaned_word

In [34]:
most_freq_word_column_labels = []
for idx in range(1, 11):
    most_freq_word_column_labels.append('most_freq_word_' + str(idx))
    
most_freq_word_column_labels

['most_freq_word_1',
 'most_freq_word_2',
 'most_freq_word_3',
 'most_freq_word_4',
 'most_freq_word_5',
 'most_freq_word_6',
 'most_freq_word_7',
 'most_freq_word_8',
 'most_freq_word_9',
 'most_freq_word_10']

In [35]:
most_freq_words_all_books = dict()
most_freq_words_all_books_df = pd.DataFrame(columns=most_freq_word_column_labels)

for book_id in top_reviews.index.unique():
    total_count_Counter = Counter()
    for index, review in top_reviews[top_reviews.index == book_id].iterrows():
        words_in_review_list = str(review['review_text']).split(' ')
        cleaned_words_in_review = filter(lambda word: (word not in set_stopwords) and (word != ''), map(clean_word, words_in_review_list))
        cleaned_review_word_count_Counter = Counter(cleaned_words_in_review)
        total_count_Counter = Counter(total_count_Counter) + Counter(cleaned_review_word_count_Counter)

    most_freq_words = total_count_Counter.most_common(10)
    most_freq_words_all_books[book_id] = most_freq_words
    
    most_freq_words_list = [word_to_freq[0] for word_to_freq in most_freq_words]
    while len(most_freq_words_list) < 10: most_freq_words_list.append(np.nan)

    most_freq_words_all_books_df.loc[book_id] = most_freq_words_list

most_freq_words_all_books_df[:10]

,most_freq_word_1,most_freq_word_2,most_freq_word_3,most_freq_word_4,most_freq_word_5,most_freq_word_6,most_freq_word_7,most_freq_word_8,most_freq_word_9,most_freq_word_10
15759838,poems,life's,rhythms,review,well,full,want,felt,reading,feel
18634307,beta,lift,one,rollo,said,like,written,men,he's,would
31921810,commentary,ancient,text,modern,enjoyable,quick,anyone,interested,spirituality,philosophy
26345942,novel,good,vanna,cheating,honest,reading,soon,could,witty,desire
17347049,end,able,spoilers,finally,find,lucifer,jack,back,mimi,human
581607,writing,i'm,entries,actual,like,woolf's,husband,great,job,selecting
25026385,und,der,das,die,reihe,ich,auch,ist,eine,hat
26237590,cady,mystery,really,like,review,doyle,done,love,historical,character
9699781,di,che,e,il,un,per,una,periodo,volta,natale
10900312,198/8th/2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
set_freq_words = set()
for col in most_freq_word_column_labels:
    set_freq_words = set_freq_words.union(set(most_freq_words_all_books_df[col].unique()))
    
    
all_most_freq_words_df = pd.DataFrame(list(set_freq_words), columns=['most_freq_words'])
all_most_freq_words_df.dropna(inplace=True)
all_most_freq_words_df

,most_freq_words
1,dansk
2,austen's
3,four
4,tv
5,ich
...,...
673,reviews
674,lewis
675,sad
676,overstretched


In [37]:
# save the most frequent words as a csv
all_most_freq_words_df.to_csv('all_most_freq_words_df.csv')

**Repeat process for the full set of 10,000 books**

In [38]:
# compile all reviews for 10,000 books 
reviews_list = top_reviews_10000['review_text'].tolist()

In [39]:
most_freq_word_column_labels = []
for idx in range(1, 11):
    most_freq_word_column_labels.append('most_freq_word_' + str(idx))
    
most_freq_word_column_labels

['most_freq_word_1',
 'most_freq_word_2',
 'most_freq_word_3',
 'most_freq_word_4',
 'most_freq_word_5',
 'most_freq_word_6',
 'most_freq_word_7',
 'most_freq_word_8',
 'most_freq_word_9',
 'most_freq_word_10']

In [40]:
most_freq_words_all_books_10000 = dict()
most_freq_words_all_books_df_10000 = pd.DataFrame(columns=most_freq_word_column_labels)

for book_id in top_reviews_10000.index.unique():
    total_count_Counter = Counter()
    for index, review in top_reviews_10000[top_reviews_10000.index == book_id].iterrows():
        words_in_review_list = str(review['review_text']).split(' ')
        cleaned_words_in_review = filter(lambda word: (word not in set_stopwords) and (word != ''), map(clean_word, words_in_review_list))
        cleaned_review_word_count_Counter = Counter(cleaned_words_in_review)
        total_count_Counter = Counter(total_count_Counter) + Counter(cleaned_review_word_count_Counter)

    most_freq_words = total_count_Counter.most_common(10)
    most_freq_words_all_books_10000[book_id] = most_freq_words
    
    most_freq_words_list = [word_to_freq[0] for word_to_freq in most_freq_words]
    while len(most_freq_words_list) < 10: most_freq_words_list.append(np.nan)

    most_freq_words_all_books_df_10000.loc[book_id] = most_freq_words_list

most_freq_words_all_books_df_10000[:10]

,most_freq_word_1,most_freq_word_2,most_freq_word_3,most_freq_word_4,most_freq_word_5,most_freq_word_6,most_freq_word_7,most_freq_word_8,most_freq_word_9,most_freq_word_10
823653,club,camel,series,first,men,oliver,stone,end,looking,plot
22046993,novella,well,alex,one,gj,wonderful,amazing,full,much,reading
17315391,e,di,che,il,non,la,per,si,anche,nel
14271,stories,writing,one,back,want,i'm,macleod,beautiful,think,feel
714382,techniques,trims,section,nice,would,interesting,beads,weaving,completed,design
1281730,little,journal,artists,well,pictures,would,life,times,great,maybe
26815133,love,reading,erotic,one,disappoint,follows,anna,two,workplace,buddies
8575158,one,slight,improvement,nice,see,character,development,jardir,big,chunk
22624779,mother,love,elisa,life,never,night,know,times,world,say
1217489,certainly,always,loved,reading,fables,kid,permeated,way,childhood,even


In [41]:
set_freq_words = set()
for col in most_freq_word_column_labels:
    set_freq_words = set_freq_words.union(set(most_freq_words_all_books_df_10000[col].unique()))
    
    
all_most_freq_words_df_10000 = pd.DataFrame(list(set_freq_words), columns=['most_freq_words'])
all_most_freq_words_df_10000.dropna(inplace=True)
all_most_freq_words_df_10000

,most_freq_words
1,wright's
2,bowl
3,cantra
4,tudo
5,rinpoche
...,...
18064,rebuttal
18065,oldalnal
18066,polow
18067,deanna


In [42]:
# save the most frequent words as a csv
all_most_freq_words_df_10000.to_csv('all_most_freq_words_df_10000.csv')